In [27]:
import json
from time import sleep

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import pyautogui as pg

In [28]:
def selenium_run():
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    url = 'https://code4biz.slack.com/'

    # 暗黙的な待機
    driver.implicitly_wait(5)
    driver.get(url)
    driver.maximize_window()
    return driver

In [29]:
def auto_login(driver):
    # ログインページ処理
    form = driver.find_element(by=By.CSS_SELECTOR, value='#signin_form')
    login_mail = form.find_element(by=By.NAME, value='email')
    login_passwd = form.find_element(by=By.NAME, value='password')

    # 認証情報
    with open('src/auth.json') as f:
        auth = json.load(f)

    my_mail = auth['my_mail']
    my_pass = auth['my_pass']

    # ログイン情報初期化
    login_mail.clear()
    login_passwd.clear()

    login_mail.send_keys(my_mail)
    login_passwd.send_keys(my_pass)

    # サインインボタンを押す
    sleep(0.5)
    btn = form.find_element(by=By.CSS_SELECTOR, value='#signin_btn')
    btn.click()
    sleep(2)

    # Slackアプリ起動確認メッセージのキャンセル
    pg.press('enter', presses=1, interval=0.1)

    # ブラウザでSlack起動
    pg.press('tab', presses=2, interval=0.1)
    pg.press('enter', presses=1, interval=0.1)
    sleep(2)

In [30]:
driver = selenium_run()
auto_login(driver)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\sgwya\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [33]:
sidebar = driver.find_element(by=By.CSS_SELECTOR, value='#channel_sidebar_group_expanded > div.p-channel_sidebar.p-channel_sidebar--classic_nav.p-channel_sidebar--iap1.p-channel_sidebar--dimming.p-channel_sidebar--contain > div > div:nth-child(1) > div > div > div.c-scrollbar__hider > div > div')
sidebar

<selenium.webdriver.remote.webelement.WebElement (session="47b7cd87b24b26f3567119d06ed68683", element="ff99b63f-d2d9-4142-a504-33a6f44c6fc8")>

In [ ]:
#C036UAP19V0 > div